In [204]:
import os
import shutil
import keras
import numpy as np
import tensorflow
import csv
import matplotlib.pyplot as plt
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [205]:
#dataset- directory original dt
original_dataset_dir = 'C:/Users/jshk/NeuralTest/train/train'
#train label
original_label_dir = 'C:/Users/jshk/NeuralTest/train/train.truth.csv'
# test data
teste_dir ='C:/Users/jshk/NeuralTest/test/test'
base_dir ='C:/Users/jshk/Test/small'

In [206]:
validation_dir = os.path.join(base_dir, 'validation')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)
validation_faces_dir = os.path.join(validation_dir, 'faces')
test_faces_dir = os.path.join(test_dir, 'faces')

In [207]:
x_train = os.path.join(original_dataset_dir)
df = pd.read_csv(original_label_dir)
df["label"] = df["label"].apply(lambda x:x.split(","))
x_test = os.path.join(teste_dir)

In [208]:
y_train = df["label"]
y_train

0         [rotated_left]
1         [rotated_left]
2              [upright]
3        [rotated_right]
4          [upside_down]
5         [rotated_left]
6         [rotated_left]
7         [rotated_left]
8          [upside_down]
9              [upright]
10         [upside_down]
11        [rotated_left]
12         [upside_down]
13       [rotated_right]
14        [rotated_left]
15             [upright]
16         [upside_down]
17         [upside_down]
18         [upside_down]
19        [rotated_left]
20       [rotated_right]
21        [rotated_left]
22        [rotated_left]
23             [upright]
24       [rotated_right]
25             [upright]
26         [upside_down]
27        [rotated_left]
28         [upside_down]
29             [upright]
              ...       
48866          [upright]
48867     [rotated_left]
48868    [rotated_right]
48869    [rotated_right]
48870    [rotated_right]
48871    [rotated_right]
48872          [upright]
48873      [upside_down]
48874    [rotated_right]


In [235]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [236]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df[:40000],
    directory=x_train,
    x_col="fn",
    y_col="label",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes=["upright", "rotated_left", "rotated_right", "upside_down"],
    target_size=(255,255))

Found 40000 validated image filenames belonging to 4 classes.


In [237]:
valid_generator=test_datagen.flow_from_directory(
    validation_dir,
    batch_size=32,
    class_mode="binary",
    target_size=(255,255))

Found 2500 images belonging to 1 classes.


In [239]:
#test_generator = test_datagen.flow_from_directory(
#    test_dir,
#    target_size=(255, 255),
#    batch_size=32,
#    class_mode='binary')
test_generator=test_datagen.flow_from_dataframe(
    dataframe=df[:5000],
    directory=x_test,
    x_col="fn",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    classes=["upright", "rotated_left", "rotated_right", "upside_down"],
    target_size=(100,100))

Found 0 validated image filenames.


C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:211: UserWarning: `classes` will be ignored given the class_mode="None"
  .format(self.class_mode))
C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 5000 invalid image filename(s) in x_col="fn". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [247]:
for data_batch, labels_batch in train_generator:
     print('data batch shape:', data_batch.shape)
     print('labels batch shape:', labels_batch.shape)
     break

data batch shape: (32, 255, 255, 3)
labels batch shape: (32, 4)


In [248]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(255, 255, 3)))

In [249]:
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(4, activation='sigmoid'))

In [250]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 253, 253, 32)      896       
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 28, 28, 128)       36992     
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 12, 12, 128)       147584    
__________

In [251]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=0.0001, decay=1e-6),
metrics=['accuracy', 'binary_crossentropy'])

In [252]:
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=30,
validation_data=test_generator,
validation_steps=50)

Epoch 1/30
100/100 [==============================] - 40s 397ms/step - loss: 0.5672 - acc: 0.7480 - binary_crossentropy: 0.5672
Epoch 2/30
100/100 [==============================] - 39s 394ms/step - loss: 0.5328 - acc: 0.7520 - binary_crossentropy: 0.5328
Epoch 3/30
100/100 [==============================] - 40s 395ms/step - loss: 0.4508 - acc: 0.7972 - binary_crossentropy: 0.4508
Epoch 4/30
100/100 [==============================] - 39s 392ms/step - loss: 0.3809 - acc: 0.8360 - binary_crossentropy: 0.3809
Epoch 5/30
100/100 [==============================] - 39s 392ms/step - loss: 0.3244 - acc: 0.8662 - binary_crossentropy: 0.3244
Epoch 6/30
100/100 [==============================] - 39s 393ms/step - loss: 0.2856 - acc: 0.8864 - binary_crossentropy: 0.2856
Epoch 7/30
100/100 [==============================] - 39s 394ms/step - loss: 0.2594 - acc: 0.8945 - binary_crossentropy: 0.2594
Epoch 8/30
100/100 [==============================] - 39s 393ms/step - loss: 0.2296 - acc: 0.9097 - bina

In [253]:
model.save('neuralTest.h5')

In [259]:
save_dir = os.path.join(os.getcwd(), 'model_save')
os.path.isdir(save_dir)
os.makedirs(save_dir)
model_path = os.path.join(save_dir, 'model_save')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\jshk\Test\model_save\model_save 
